# Diff-Taichi Experiments for Physics engines

In [1]:
import taichi as ti
import sys 
import math
import numpy as np
import os 
import matplotlib.pyplot as plt

[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.7.16


[I 03/27/25 23:48:49.918 469425] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
'''
If the ti.gpu option is specified, Taichi will attempt to use the GPU backends in the following order: 
ti.cuda, ti.vulkan, and ti.opengl/ti.Metal. 

If no GPU architecture is available, the CPU will be used as the backend.
'''

real = ti.f32 # float 32
ti.init(default_fp=real, flatten_if=True, arch=ti.gpu) # default arch is cpu, flatten_if is used to flatten if statement

[Taichi] Starting on arch=cuda


In [3]:
max_steps = 2048
vis_interval = 64
output_vis_interval = 16
steps = 1024
assert steps * 2 <= max_steps

vis_resolution = 1024

In [4]:
scalar = lambda: ti.field(dtype=real) # scalar field
vec = lambda: ti.Vector.field(2, dtype=real) # vector field

loss = scalar()
init_x = vec()
init_v = vec()

x = vec()
x_inc = vec()  # for TOI
v = vec()
impulse = vec()

In [5]:
billiard_layers = 4
n_balls = 1 + (1 + billiard_layers) * billiard_layers // 2
target_ball = n_balls - 1
# target_ball = 0
goal = [0.9, 0.75]
radius = 0.03
elasticity = 0.8

In [6]:
'''
Notes on Taichi's hierarchical datastructures can be found on obsidian

https://taichi.readthedocs.io/en/stable/hierarchical.html

Here, we define the data structure for the hierarchical simulation

Root - the root of the simulation
root.dense - max_steps number of time steps (nodes)
root.dense.dense - each timestep has n_balls number of balls (nodes)
root.dense.dense.place - each ball has a position, velocity, position increment, and impulse (leaf nodes)

root.place - has 3 leaf values - (initial position and velocity of the balls) and (loss)
'''

ti.root.dense(ti.i, max_steps).dense(ti.j, n_balls).place(x, v, x_inc, impulse)
ti.root.place(init_x, init_v)
ti.root.place(loss)
ti.root.lazy_grad() # The gradients are computed only when needed (most likely)

In [7]:
dt = 0.003
alpha = 0.00000
learning_rate = 0.01

In [8]:
@ti.func
def collide_pair(t, i, j):
    imp = ti.Vector([0.0, 0.0])
    x_inc_contrib = ti.Vector([0.0, 0.0])
    if i != j:
        dist = (x[t, i] + dt * v[t, i]) - (x[t, j] + dt * v[t, j])
        dist_norm = dist.norm()
        rela_v = v[t, i] - v[t, j]
        if dist_norm < 2 * radius: # In collision
            dir = ti.Vector.normalized(dist, 1e-6)
            projected_v = dir.dot(rela_v) 

            if projected_v < 0: # Implies that the objects are moving towards each other
                imp = -(1 + elasticity) * 0.5 * projected_v * dir # Calculate impulse for each object
                toi = (dist_norm - 2 * radius) / min(
                    -1e-3, projected_v)  # Time of impact
                x_inc_contrib = min(toi - dt, 0) * imp # Change in position increment
    x_inc[t + 1, i] += x_inc_contrib
    impulse[t + 1, i] += imp

In [9]:

@ti.kernel
def collide(t: ti.i32):
    for i in range(n_balls):
        for j in range(i):
            collide_pair(t, i, j)
    for i in range(n_balls):
        for j in range(i + 1, n_balls):
            collide_pair(t, i, j)


@ti.kernel
def advance(t: ti.i32):
    for i in range(n_balls):
        v[t, i] = v[t - 1, i] + impulse[t, i]
        x[t, i] = x[t - 1, i] + dt * v[t, i] + x_inc[t, i]


@ti.kernel
def compute_loss(t: ti.i32):
    loss[None] = (x[t, target_ball][0] - goal[0])**2 + (x[t, target_ball][1] -
                                                        goal[1])**2


@ti.kernel
def initialize():
    x[0, 0] = init_x[None]
    v[0, 0] = init_v[None]


gui = ti.GUI("Billiards", (1024, 1024), background_color=0x3C733F)


def forward(visualize=False, output=None):
    initialize()

    interval = vis_interval
    if output:
        interval = output_vis_interval
        os.makedirs('billiards/{}/'.format(output), exist_ok=True)

    count = 0
    for i in range(billiard_layers):
        for j in range(i + 1):
            count += 1
            x[0, count] = [
                i * 2 * radius + 0.5, j * 2 * radius + 0.5 - i * radius * 0.7
            ]

    pixel_radius = int(radius * 1024) + 1

    for t in range(1, steps):
        collide(t - 1)
        advance(t)

        if (t + 1) % interval == 0 and visualize:
            gui.clear()
            gui.circle((goal[0], goal[1]), 0x00000, pixel_radius // 2)

            for i in range(n_balls):
                if i == 0:
                    color = 0xCCCCCC
                elif i == n_balls - 1:
                    color = 0x3344cc
                else:
                    color = 0xF20530

                gui.circle((x[t, i][0], x[t, i][1]), color, pixel_radius)

            if output:
                gui.show('billiards/{}/{:04d}.png'.format(output, t))
            else:
                gui.show()

    compute_loss(steps - 1)


@ti.kernel
def clear():
    for t, i in ti.ndrange(max_steps, n_balls):
        impulse[t, i] = ti.Vector([0.0, 0.0])
        x_inc[t, i] = ti.Vector([0.0, 0.0])


def optimize():
    init_x[None] = [0.1, 0.5]
    init_v[None] = [0.3, 0.0]

    clear()
    # forward(visualize=True, output='initial')

    for iter in range(200):
        clear()

        with ti.ad.Tape(loss):
            if iter % 20 == 19:
                output = 'iter{:04d}'.format(iter)
            else:
                output = None
            forward(visualize=True, output=output)

        print('Iter=', iter, 'Loss=', loss[None])
        for d in range(2):
            init_x[None][d] -= learning_rate * init_x.grad[None][d]
            init_v[None][d] -= learning_rate * init_v.grad[None][d]

    clear()
    forward(visualize=True, output='final')


def scan(zoom):
    '''
    Scans the objective function with respect to the angle of the initial velocity
    The objective function is the distance between the target ball and the goal
    The angle of the initial velocity is varied from -pi/2 to pi/2
    The initial position of the ball is fixed at (0.1, 0.5)
    The initial velocity is varied from 0 to 0.3
    The objective function is plotted against the angle of the initial velocity
    The plot shows that the objective function is minimized when the angle of the initial velocity is 0
    '''
    N = 1000
    angles = []
    losses = []
    forward(visualize=True, output='initial')
    for i in range(N):
        alpha = ((i + 0.5) / N - 0.5) * math.pi * zoom
        init_x[None] = [0.1, 0.5]
        init_v[None] = [0.3 * math.cos(alpha), 0.3 * math.sin(alpha)]

        loss[None] = 0
        clear()
        forward(visualize=False)
        print(loss[None])

        losses.append(loss[None])
        angles.append(math.degrees(alpha))

    plt.plot(angles, losses)
    fig = plt.gcf()
    fig.set_size_inches(5, 3)
    plt.title('Billiard Scene Objective')
    plt.ylabel('Objective')
    plt.xlabel('Angle of velocity')
    plt.tight_layout()
    plt.show()


# if __name__ == '__main__':
#     if len(sys.argv) > 1:
#         scan(float(sys.argv[1]))
#     else:
        # optimize()

In [10]:
optimize()

[W 03/27/25 23:49:13.672 469596] TaichiWarning
While compiling `collide_c77_0_reverse_grad`, File "/home/vishal/anaconda3/envs/disect_3_7/lib/python3.7/site-packages/taichi/lang/matrix_ops.py", line 27, in _reduce:
        result = mat[0]
Loading variable 114 before anything is stored to it.
[W 03/27/25 23:49:13.672 469596] TaichiWarning
While compiling `collide_c77_0_reverse_grad`, File "/home/vishal/anaconda3/envs/disect_3_7/lib/python3.7/site-packages/taichi/lang/matrix_ops.py", line 29, in _reduce:
            result = fun(result, mat[i])
Loading variable 117 before anything is stored to it.
[W 03/27/25 23:49:13.672 469596] TaichiWarning
While compiling `collide_c77_0_reverse_grad`, File "/home/vishal/anaconda3/envs/disect_3_7/lib/python3.7/site-packages/taichi/lang/matrix_ops.py", line 27, in _reduce:
        result = mat[0]
Loading variable 136 before anything is stored to it.
[W 03/27/25 23:49:13.672 469596] TaichiWarning
While compiling `collide_c77_0_reverse_grad`, File "/home

Iter= 0 Loss= 0.045418091118335724
Iter= 1 Loss= 0.029302814975380898
Iter= 2 Loss= 0.0287479255348444
Iter= 3 Loss= 0.025674356147646904
Iter= 4 Loss= 0.023475026711821556
Iter= 5 Loss= 0.022154800593852997
Iter= 6 Loss= 0.019496990367770195
Iter= 7 Loss= 0.01793975941836834
Iter= 8 Loss= 0.01646742969751358
Iter= 9 Loss= 0.014963734894990921
Iter= 10 Loss= 0.013615012168884277
Iter= 11 Loss= 0.012929102405905724
Iter= 12 Loss= 0.011220036074519157
Iter= 13 Loss= 0.010551296174526215
Iter= 14 Loss= 0.009644299745559692
Iter= 15 Loss= 0.009062526747584343
Iter= 16 Loss= 0.00817928183823824
Iter= 17 Loss= 0.007591473404318094
Iter= 18 Loss= 0.006654092576354742
Iter= 19 Loss= 0.005971149541437626
Iter= 20 Loss= 0.005608173552900553
Iter= 21 Loss= 0.0051395767368376255
Iter= 22 Loss= 0.004673346411436796
Iter= 23 Loss= 0.004225767217576504
Iter= 24 Loss= 0.0039317118935287
Iter= 25 Loss= 0.003359076101332903
Iter= 26 Loss= 0.003300844458863139
Iter= 27 Loss= 0.003307668725028634
Iter= 28

KeyboardInterrupt: 

In [11]:
gui.close()